In [27]:
import os
from pyrms import rms
from julia import Main
import time 
import yaml
import pandas as pd
import math
from copy import deepcopy
from rmgpy.chemkin import load_chemkin_file
from rmgpy.rmg.model import ReactionModel
from rmgpy.species import Species
from rmgpy.kinetics import StickingCoefficientBEP, StickingCoefficient, SurfaceArrheniusBEP, SurfaceArrhenius
from rmgpy.data.kinetics.database import KineticsDatabase
from matplotlib import pyplot as plt
%matplotlib inline

In [186]:
def make_spc(spc):
    """
    make an RMG object from the rms object
    
    """
    if len(spc.adjlist) > 0:
        rmg_spc = Species().from_adjacency_list(spc.adjlist)
    else:
        rmg_spc = Species().from_smiles(spc.smiles)

    return rmg_spc

def initial_val_unc(kinetics, num): 
    """ 
    return the initial value, an estimate of parameter uncertainty, and upper
     as a tuple
    A-factors: if not sticking coeffient, +/- 1, return LogA. else, +/- 0.5
    Ea: +/- 0.3 eV. 

    Limits: 
    A (upper):  - 1 for sticking coefficient, 1e25 for others
    A (lower):  - 0 for sticking coefficient, 0 for others
    E (upper):  - 400 kJ/mol
    E (lower):  - 0 kj/mol
    """ 
    # logA if it is not sticking coefficient
    if kinetics["A"] > 1: 
        A = math.log10(kinetics["A"])
        A_unc = 1
        A_lb = 0
        A_ub = 25
        label_A = f"A_log_{num}"
    elif kinetics["A"] <= 1: 
        A = kinetics["A"]
        A_unc = 1
        A_lb = 0
        A_ub = 1
        label_A = f"A_stick_{num}"
    
    E = kinetics["Ea"]
    E_unc = 30000000 # J/kmol for cantera
    E_lb = 0
    E_ub = 400000000 # J/kmol
    label_E = f"E_{num}"
    
    return (A, E), (A_unc, E_unc), (A_lb, E_lb), (A_ub, E_ub), (label_A, label_E)

In [13]:
base_path = "/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/rmg_gua/baseline"
# base_path = "/Users/blais.ch/Documents/_01_code/05_Project_repos_Github/meOH_repos/uncertainty_analysis/rmg_gua/baseline"

# load rms model
rms_path = os.path.join(base_path, "rms", "chem32.rms")
phase_dict = rms.readinput(rms_path)

# load chemkin files
ck_path = os.path.join(base_path, "chemkin")
cmkn_path = os.path.join(ck_path, "chem_annotated-gas.inp")
cmkn_surf_path = os.path.join(ck_path, "chem_annotated-surface.inp")
cmkn_dict_path = os.path.join(ck_path, "species_dictionary.txt")

In [14]:
# get species from rms model
gas_spec = phase_dict["gas"]["Species"]
surf_spec = phase_dict["surface"]["Species"]

In [15]:
# load the chemkin file for the mechanism
model = ReactionModel()

model.species, model.reactions = load_chemkin_file(
    cmkn_path,
    cmkn_dict_path,
    surface_path=cmkn_surf_path,
)

In [16]:
# make a dict of rms species with name as key, rmg obj as value
rms_spc_dict = {}
for spc in gas_spec:
    rmg_spc = make_spc(spc)
    rms_spc_dict[spc.name] = rmg_spc
for spe in surf_spec:
    rmg_spc = make_spc(spe)
    rms_spc_dict[spe.name] = rmg_spc

In [17]:
# now make the species for the chemkin mechanism. 
# match each species string with it's chemkin counterpart
rmg_2_ck_dict = {}
for species in model.species:
    for rms_name, spec in rms_spc_dict.items():
        if species.is_isomorphic(spec):
            ck_name = species.to_cantera(use_chemkin_identifier=True).name
            rmg_2_ck_dict[rms_name] = ck_name
with open("rmg_2_ck_dict.yaml", "w") as f:
    yaml.dump(rmg_2_ck_dict, f)

In [18]:
# now load the sensitive reactions
results_path = "/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/rmg_gua/baseline/rms/rms_analysis.csv"
res_df = pd.read_csv(results_path)
res_df = res_df[res_df["use_for_opt"] == True]

## get most sensitive values for sens_species

In [19]:
# get columns for each secies sensitivity
scols = {
    "CH3OH" : res_df.filter(like='sens to CH3OH').columns, 
    "CC" : res_df.filter(like='sens to CC').columns,
    "CH4" : res_df.filter(like='sens to CH4').columns, 
}
res_df[scols["CH3OH"][0:5]]

,H*+CH3OH*<=>H2O*+CH3X sens to CH3OH,H*+OC[Pt]<=>X+CH3OH* sens to CH3OH,COOH*+CH3OH*<=>HCOOH*+OC[Pt] sens to CH3OH,H*+CH2O*<=>X+OC[Pt] sens to CH3OH,HCOOH*+OC[Pt]<=>HCOO*+CH3OH* sens to CH3OH
0,-1390.102036,1371.853201,11.609749,5.636080,5.049337
2,-1942.132908,7.667782,0.141083,1849.941893,0.060971
3,-2403.045204,-3.039584,-0.077219,2306.882661,-0.034080
6,-155.215975,155.652313,0.565829,0.019619,0.244928
7,-1152.425221,890.898034,3.126900,262.177569,1.353510
...,...,...,...,...,...
195,-9342.795846,5634.400331,1.892261,3685.272035,0.818775
197,-9056.234444,9054.927493,0.843093,30.349014,0.364869
198,-10260.827065,10256.661224,1.711249,35.585819,0.740554
199,-11693.299174,11685.561908,2.726504,41.826941,1.179868


### we want the reactions for each that are most sensitive over all experiments 

In [20]:
with open("../gua_cantera/all_experiments_reorg_sbr.yaml", "r") as f:
    expt_all = yaml.load(f, Loader=yaml.FullLoader)

opt_expt_file = "../gua_cantera/experiments_reorg_onlyopt.yaml"
expt_copy = []
for expt in expt_all:
    if "use_for_opt" in expt.keys() and expt["use_for_opt"] == True:
        expt_copy.append(expt)

with open(opt_expt_file, "w") as f:
    yaml.dump(expt_copy, f)
    

In [21]:
# there are a number of empty cols, pd.dropna
highest_sens_dict = {}
for names, cols in scols.items():
    
    res_df_T = res_df[cols].T
    sum_dict = {}
    for name, row in res_df_T.iterrows():
        total = row.abs().sum()
        sum_dict[name] = total

    # sort dict by value to get most sensitive
    highest = max(sum_dict, key=sum_dict.get)
    highest_sens_dict[names] = [highest, float(sum_dict[highest])]
highest_sens_dict

with open("highest_sens_rms.yaml", "w") as f:
    yaml.dump(highest_sens_dict, f)

In [22]:
sens_rxn_str = highest_sens_dict["CH3OH"][0].split(" ")[0]
sens_reac = sens_rxn_str.split("<=>")[0].split("+")
sens_prod = sens_rxn_str.split("<=>")[1].split("+")

In [23]:
with open("rmg_2_ck_dict.yaml", "r") as f: 
    rms_2_ck = yaml.load(f, Loader=yaml.FullLoader)
    
sens_reac_ct = [rms_2_ck[name] for name in sens_reac]
sens_prod_ct = [rms_2_ck[name] for name in sens_prod]
sens_reac_ct, sens_prod_ct

(['H*(10)', 'CH3OH*(23)'], ['H2O*(13)', 'CH3X(33)'])

In [24]:
! source activate ../../peuquse_env ; cti2yaml "../baseline/cantera/chem_annotated.cti" 

Wrote YAML mechanism file to '../baseline/cantera/chem_annotated.yaml'.
Mechanism contains 32 species and 3 reactions.
Validating mechanism...
/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/peuquse_env/lib/python3.7/site-packages/cantera/cti2yaml.py:1784: UserWarning: StickingRate::validate: 
 Sticking coefficient is greater than 1 for reaction 'CC(134) + 2 X(1) <=> C2H5X(136) + H*(10)'
 at T = 10000.0

  phase = ct.Interface(output_name, surf_name)
PASSED


In [1]:
import cantera as ct
gas = ct.Solution( "../baseline/cantera/chem_annotated.cti", "gas")
surf = ct.Interface("../baseline/cantera/chem_annotated.cti" , "surface1", [gas])

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/peuquse_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: XML_Node::build: 
The CTI and XML input file formats are deprecated and will be removed in
Cantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input
files to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html
for more information.
  
/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/peuquse_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: InterfaceReaction2::validate: 
 Sticking coefficient is greater than 1 for reaction 'CC(134) + 2 X(1) <=> C2H5X(136) + H*(10)'
 at T = 10000.0

  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
surf.reactions()[0].rate


Arrhenius(A=0.046, b=0, E=0)

In [3]:
sens_reac_ct

NameError: name 'sens_reac_ct' is not defined

In [ ]:
reactants

In [187]:
# with open("../baseline/cantera/chem_annotated.yaml", "r") as f: 
#     ct_mech = yaml.load(f, Loader=yaml.FullLoader)

import collections

ct_reac = collections.Counter(sens_reac_ct)
ct_prod = collections.Counter(sens_prod_ct)

# get the most sensitive reaction from cti mech
for num, rxn in enumerate(surf.reactions()): 
    
    reaclist = []
    prodlist = []
    for reac, stoic in rxn.reactants.items():
        if stoic > 1: 
            reaclist.extend([reac]*int(stoic))
        else: 
            reaclist.append(reac)
            
    for reac, stoic in rxn.products.items():
        if stoic > 1: 
            prodlist.extend([reac]*int(stoic))
        else: 
            prodlist.append(reac)
    
    
    reactants = collections.Counter(reaclist)
    products = collections.Counter(prodlist)
    
    fwd = reactants == ct_reac and products == ct_prod
    rev = reactants == ct_prod and products == ct_reac

    if rev or fwd: 

        reac_dict = {}
        reac_dict["A"] = rxn.rate.pre_exponential_factor
        reac_dict["Ea"] = rxn.rate.activation_energy
        
        (A, E), (A_unc, E_unc), (A_lb, E_lb), (A_ub, E_ub), (label_A, label_E ) =  initial_val_unc(reac_dict, num)

In [188]:
ct_peuqyml = {}
ct_peuqyml["initial_values"] = [A, E]
ct_peuqyml["labels"] = [label_A, label_E]
ct_peuqyml["lower_bounds"] = [A_lb, E_lb]
ct_peuqyml["uncertainties"] = [A_unc, E_unc]
ct_peuqyml["upper_bounds"] = [A_ub, E_ub]

with open("ct_initial_small.yaml", "w") as f:
    yaml.dump(ct_peuqyml, f)

In [117]:
ct_mech["surface1-reactions"][149]

{'equation': 'H*(10) + CH3OH*(23) <=> H2O*(13) + CH3X(33)',
 'id': 'surface1-150',
 'rate-constant': {'A': 1e+17, 'b': 0.0, 'Ea': 0.0}}

In [37]:
import cantera as ct

In [38]:
! pwd

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/rmg_gua/gua_peuqse


In [93]:
rate = ct.Arrhenius(A = 2.0, 
                    E = 2, 
                   b = 0)
new_rxn.rate = rate
new_rxn.rate

Arrhenius(A=2, b=0, E=2)

In [4]:
gas = ct.Solution("../baseline/cantera/chem_annotated.cti")
surf = ct.Interface("../baseline/cantera/chem_annotated.cti", "surface1", [gas])

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/peuquse_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: InterfaceReaction2::validate: 
 Sticking coefficient is greater than 1 for reaction 'CC(134) + 2 X(1) <=> C2H5X(136) + H*(10)'
 at T = 10000.0

  


In [7]:
surf.reactions()[149].rate

Arrhenius(A=1e+16, b=0, E=0)

In [8]:
num = 0
A = 1e17
Ea = 2.0
new_rxn = surf.reactions()[num]

rate = ct.Arrhenius(A = A, 
                    E = Ea, 
                   b = 0
                   )
new_rxn.rate = rate


In [9]:
surf.modify_reaction(num, new_rxn)

In [10]:
surf.reactions()[0].rate

Arrhenius(A=1e+17, b=0, E=2)

In [126]:
surf.reactions()[0].reactants

{'H2(2)': 1.0, 'X(1)': 2.0}

In [11]:
len(surf.reactions())

179